In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from model import PIC

In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize pixel values
])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

In [27]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1) # 1 входной канал, 32 выходных канала
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(7*7*16, 10)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.fc1(x)
        return x

In [28]:
model = SimpleCNN().to(DEVICE)

In [29]:
LEARNING_RATE = 0.001
EPOCHS = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [30]:
print("Start training...")
for epoch in range(EPOCHS):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {running_loss/len(train_loader):.4f}")
    running_loss = 0.0

# 6. Оценка модели
print("Finished training...")   

Start training...
Epoch [1/10] Loss: 0.1815
Epoch [2/10] Loss: 0.0695
Epoch [3/10] Loss: 0.0522
Epoch [4/10] Loss: 0.0428
Epoch [5/10] Loss: 0.0363
Epoch [6/10] Loss: 0.0320
Epoch [7/10] Loss: 0.0278
Epoch [8/10] Loss: 0.0256
Epoch [9/10] Loss: 0.0221
Epoch [10/10] Loss: 0.0204
Finished training...


In [31]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Get predicted classes
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

print("End of experiment")

Accuracy on the test set: 98.81%
End of experiment


In [47]:
class SimplePIC(nn.Module):
    def __init__(self):
        super(SimplePIC, self).__init__()
        self.pic1 = PIC(1, 8, 8, 3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.pic2 = PIC(8, 16, 16, 3)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(7*7*16, 10)

    def forward(self, x):
        x = self.maxpool1(self.pic1(x))
        x = self.maxpool2(self.pic2(x))
        x = self.flatten(x)
        x = self.fc1(x)
        return x

In [48]:
pic = SimplePIC().to(DEVICE)

In [49]:
LEARNING_RATE = 0.001
EPOCHS = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pic.parameters(), lr=LEARNING_RATE)

In [50]:
print("Start training...")
for epoch in range(EPOCHS):
    pic.train()  # Set model to training mode
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прямой проход
        outputs = pic(images)
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {running_loss/len(train_loader):.4f}")
    running_loss = 0.0

# 6. Оценка модели
print("Finished training...")   

Start training...
Epoch [1/10] Loss: 0.4506
Epoch [2/10] Loss: 0.1310
Epoch [3/10] Loss: 0.0914
Epoch [4/10] Loss: 0.0742
Epoch [5/10] Loss: 0.0634
Epoch [6/10] Loss: 0.0563
Epoch [7/10] Loss: 0.0510
Epoch [8/10] Loss: 0.0471
Epoch [9/10] Loss: 0.0434
Epoch [10/10] Loss: 0.0407
Finished training...


In [53]:
pic.eval()  # Set model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = pic(images)
        _, predicted = torch.max(outputs.data, 1)  # Get predicted classes
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

print("End of experiment")

Accuracy on the test set: 98.43%
End of experiment
